# Installation

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar100
from keras import utils
from keras.models import Sequential
from keras.layers import Dropout, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras import optimizers
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

print('Libraries imported.')

Libraries imported.


#Hyperparameters

In [2]:
#training constants
BATCH_SIZE = 128
N_EPOCH = 20
N_CLASSES = 100
VERBOSE = 1
VALIDATION_SPLIT = 5 # 5-folding cross validation

# Experienment hyperparameters
param_grid = {'lr': [0.05, 0.01, 0.005],
        'dropout': [0.1, 0.2, 0.3],
        'k_size': [(3,3), (5,5), (7,7)]}

print('Main variables initialised.')

Main variables initialised.


In [3]:
# CIFAR_100 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

print('Image variables initialisation')

Image variables initialisation


#CIFAR-100 data loading and processing

In [4]:
#load dataset
(input_X_train, output_y_train), (input_X_test, output_y_test) = cifar100.load_data()
print('input_X_train shape:', input_X_train.shape)
print(input_X_train.shape[0], 'train samples')
print(input_X_test.shape[0], 'test samples')

# convert to categorical
output_Y_train = utils.to_categorical(output_y_train, N_CLASSES)
output_Y_test = utils.to_categorical(output_y_test, N_CLASSES)

# float and normalization
input_X_train = input_X_train.astype('float32')
input_X_test = input_X_test.astype('float32')
input_X_train /= 255
input_X_test /= 255

input_X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
print(output_Y_train.shape)

(50000, 100)


#Network Definition

In [6]:
def create_model(dropout=0.1, lr=0.5, k_size=(3,3)):
    model = Sequential()

    model.add(Conv2D(192, k_size, padding='same',input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(160, (1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(96, (1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
    model.add(Dropout(dropout))

    model.add(Conv2D(192, k_size, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(192, (1, 1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(192, (1, 1),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
    model.add(Dropout(dropout))

    model.add(Conv2D(192, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(192, (1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(100, (1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))

    opt = optimizers.Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [7]:
# Wrap model in a KerasClassier
model = KerasClassifier(build_fn=create_model,verbose=VERBOSE, dropout=0.1, lr=0.05, k_size=(3,3))

#Grid Search

In [8]:
# Initialise GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=VALIDATION_SPLIT, scoring="accuracy")

# Fit GridSearchCV to your data
grid.fit(input_X_train, output_Y_train, epochs=N_EPOCH, batch_size=BATCH_SIZE, verbose=VERBOSE)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 32s 54ms/step - accuracy: 0.0680 - loss: 4.1993
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.2037 - loss: 3.3370
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.3036 - loss: 2.8235
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.3688 - loss: 2.5156
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.4177 - loss: 2.2838
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.4563 - loss: 2.1282
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.4943 - loss: 1.9648
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5327 - loss: 1.8153
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5495 - loss: 1.7322
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5752 - loss: 1.6452
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5964 - loss: 1.5446
Epoch 12/20
313/313 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 47ms/step - accuracy: 0.0622 - loss: 4.2293
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.1870 - loss: 3.4957
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.2821 - loss: 3.0423
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.3377 - loss: 2.7747
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.3919 - loss: 2.5241
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.4193 - loss: 2.3967
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.4545 - loss: 2.2550
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.4814 - loss: 2.0996
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5142 - loss: 1.9695
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5378 - loss: 1.8772
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5590 - loss: 1.7951
Epoch 12/20
313/313 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 51ms/step - accuracy: 0.0922 - loss: 4.0726
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.2073 - loss: 3.3279
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.2960 - loss: 2.8275
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.3625 - loss: 2.4948
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.4212 - loss: 2.2285
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.4660 - loss: 2.0304
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.4977 - loss: 1.8803
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.5331 - loss: 1.7474
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.5562 - loss: 1.6377
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.5878 - loss: 1.5116
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.6136 - loss: 1.4113
Epoch 12/20
313/313 ━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - accuracy: 0.1122 - loss: 3.9844
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.2830 - loss: 3.0041
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.3764 - loss: 2.4915
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.4450 - loss: 2.1542
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.4954 - loss: 1.9172
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.5410 - loss: 1.7360
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.5733 - loss: 1.5834
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.6046 - loss: 1.4614
Epoch 9/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.6264 - loss: 1.3643
Epoch 10/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.6534 - loss: 1.2615
Epoch 11/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.6762 - loss: 1.1878
Epoch 12/20
391/391 ━━━━━━━━━━

GridSearchCV(cv=5,
             estimator=KerasClassifier(build_fn=<function create_model at 0x7f1063ebe200>, dropout=0.1, k_size=(3, 3), lr=0.05),
             param_grid={'dropout': [0.1, 0.2, 0.3],
                         'k_size': [(3, 3), (5, 5), (7, 7)],
                         'lr': [0.05, 0.01, 0.005]},
             scoring='accuracy')

In [9]:
#Print Results
print(grid.cv_results_)

{'mean_fit_time': array([195.64444079, 194.1397532 , 192.4630682 , 204.64935956,
       204.24776502, 204.41090794, 239.45381732, 239.15553031,
       239.09833317, 192.74559016, 191.84563255, 192.71853228,
       205.44463005, 206.38882647, 205.57073565, 239.77529392,
       238.82415571, 239.14141598, 192.59881768, 189.96976857,
       189.26186118, 205.17125435, 204.87498927, 205.06373711,
       237.77305737, 239.85290613, 238.3355257 ]), 'std_fit_time': array([2.46743051, 0.31076845, 1.62756961, 1.2461368 , 0.42114231,
       0.69189745, 2.0414488 , 0.69595269, 0.2192839 , 1.29481839,
       0.20630195, 1.50842087, 0.66702451, 1.55114404, 0.28153596,
       2.05436304, 0.39623713, 0.2087679 , 2.17204381, 0.75932106,
       0.07166521, 2.14839806, 0.29509424, 0.04342155, 0.69676781,
       2.60948001, 0.19691281]), 'mean_score_time': array([2.29089875, 2.13213267, 2.23436151, 2.18983831, 2.12100935,
       2.1106977 , 2.34496689, 2.6779778 , 2.26516285, 2.09483271,
       2.1058455

In [10]:
print(grid.best_estimator_)

KerasClassifier(
	model=None
	build_fn=<function create_model at 0x7f1063ebe200>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	dropout=0.1
	lr=0.005
	k_size=(3, 3)
	class_weight=None
)


In [11]:
# Train the best estimator
history = grid.best_estimator_.fit(input_X_train, output_Y_train, batch_size=BATCH_SIZE, epochs=N_EPOCH, validation_split=0.2,  verbose=VERBOSE)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 51ms/step - accuracy: 0.1050 - loss: 4.0407 - val_accuracy: 0.0649 - val_loss: 4.4696
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.2451 - loss: 3.2002 - val_accuracy: 0.1780 - val_loss: 3.5358
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.3420 - loss: 2.6725 - val_accuracy: 0.3077 - val_loss: 2.7849
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.4081 - loss: 2.3301 - val_accuracy: 0.3269 - val_loss: 2.6365
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.4655 - loss: 2.0727 - val_accuracy: 0.4047 - val_loss: 2.3434
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.4962 - loss: 1.9178 - val_accuracy: 0.4262 - val_loss: 2.2096
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.5386 - loss: 1.7420 - val_accuracy: 0.3921 - val_loss: 2.4426
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.5710 - loss: 1.6022 - val_acc

In [17]:
# Test the best estimator
score = history.score(input_X_test, output_Y_test)
print(score)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
0.5367
